In [1]:
#!conda install -y geopandas folium

In [2]:
#!pip install osmnx

# Setup

In [3]:
import geopandas as gpd
import osmnx as ox
import folium
import pandas as pd
from shapely.geometry import Point

# World boundaries

In [4]:
world = gpd.read_file('./boundaries/boundaries.shp')

In [5]:
world.head()

,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,ADMIN,ADM0_A3,...,NAME_KO,NAME_NL,NAME_PL,NAME_PT,NAME_RU,NAME_SV,NAME_TR,NAME_VI,NAME_ZH,geometry
0,Admin-0 country,1,3,Zimbabwe,ZWE,0,2,Sovereign country,Zimbabwe,ZWE,...,짐바브웨,Zimbabwe,Zimbabwe,Zimbábue,Зимбабве,Zimbabwe,Zimbabve,Zimbabwe,辛巴威,"POLYGON ((31.28789 -22.40205, 31.19727 -22.344..."
1,Admin-0 country,1,3,Zambia,ZMB,0,2,Sovereign country,Zambia,ZMB,...,잠비아,Zambia,Zambia,Zâmbia,Замбия,Zambia,Zambiya,Zambia,赞比亚,"POLYGON ((30.39609 -15.64307, 30.25068 -15.643..."
2,Admin-0 country,1,3,Yemen,YEM,0,2,Sovereign country,Yemen,YEM,...,예멘,Jemen,Jemen,Iémen,Йемен,Jemen,Yemen,Yemen,也门,"MULTIPOLYGON (((53.08564 16.64839, 52.58145 16..."
3,Admin-0 country,3,2,Vietnam,VNM,0,2,Sovereign country,Vietnam,VNM,...,베트남,Vietnam,Wietnam,Vietname,Вьетнам,Vietnam,Vietnam,Việt Nam,越南,"MULTIPOLYGON (((104.06396 10.39082, 104.08301 ..."
4,Admin-0 country,5,3,Venezuela,VEN,0,2,Sovereign country,Venezuela,VEN,...,베네수엘라,Venezuela,Wenezuela,Venezuela,Венесуэла,Venezuela,Venezuela,Venezuela,委內瑞拉,"MULTIPOLYGON (((-60.82119 9.13838, -60.94141 9..."


# Pitches

## Polygon

In [6]:
country_of_choice = 'Germany'
cc = 'ge'
centre = [52, 10.2] 

In [7]:
country = world[world.ADMIN == country_of_choice]

In [8]:
poly = country.geometry.iloc[0]

## Query OSM

In [9]:
# Download all leisure footprints in our focus area
leisure = ox.footprints.footprints_from_polygon(poly, footprint_type='leisure')
leisure.loc[:,['geometry', 'leisure', 'note', 'name', 'surface']].to_file('./geojson/osm_leisure_' + cc +'.geojson', driver='GeoJSON')

MemoryError: Unable to allocate 6.44 GiB for an array with shape (1903, 454236) and data type object

## Filter for soccer pitches

In [ ]:
pitches = leisure[(leisure.leisure=='pitch') | (leisure.leisure=='stadium')]

In [ ]:
soccer = pitches[pitches.sport == 'soccer']

In [ ]:
pd.options.display.max_columns = 300

In [ ]:
soccer.info()

In [ ]:
soccer.head()

## Plot

In [ ]:
soccer.loc[:,['geometry', 'leisure', 'note', 'name', 'surface']].to_file('osm_soccer_pitches_' + cc +'.geojson', driver='GeoJSON')

In [ ]:
soccer_x = [point.x for point in soccer.geometry.centroid]
soccer_y = [point.y for point in soccer.geometry.centroid]

In [ ]:
tileset_ID_str = "satellite-v9"
mapbox_access_token = "pk.eyJ1Ijoic2VieHdvbGYiLCJhIjoiY2pvbmVlbHVvMHV5cjNrcnl3ZnU0cGU3eSJ9.jP3kUXiKwKFXwQyUgutldQ"
tilesize_pixels = '512'
mapbox_tile_URL = f"https://api.mapbox.com/styles/v1/mapbox/{tileset_ID_str}/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_access_token}"

In [ ]:
m = folium.Map(
        location=centre,
        tiles=None,
        zoom_start=8,
        control_scale = True)

folium.TileLayer('openstreetmap', name = 'OpenStreetMap').add_to(m)
folium.TileLayer(mapbox_tile_URL, attr = 'Mapbox', name = 'Satellite', overlay = False).add_to(m)

folium.GeoJson(soccer, name = 'Football Pitches', 
        style_function= lambda x: {"weight": 2, 'color': 'black', 'fillColor': 'blue', 'fillOpacity':0.8},
        highlight_function=lambda x: {'weight':4, 'color':'black'},
        smooth_factor=1,
        show = True,
        tooltip=folium.features.GeoJsonTooltip(fields = ['name', 'note', 'surface' , 'source',],
                                                  aliases=['Name', 'Note' , 'Surface', 'Source'], 
                                                  labels=True, 
                                                  sticky=True,
                                                  localize=True
                                                 )).add_to(m)

soccer_list = zip(soccer_y, soccer_x)
for lat, lon in soccer_list:
    folium.CircleMarker(location = [lat, lon], 
                        fill_color = 'green',
                        radius = 5,
                        color='g',
                        fill = True,
                        fill_opacity = 0.7).add_to(m)

folium.LayerControl().add_to(m)
    
m.save('./html_maps/osm_soccer_pitches_' + cc +'.html')

In [ ]:
m = folium.Map(
        location=centre,
        tiles=None,
        zoom_start=5,
        control_scale = True)

folium.TileLayer('openstreetmap', name = 'Map').add_to(m)
folium.TileLayer(mapbox_tile_URL, attr = 'Mapbox', name = 'Satellite', overlay = False).add_to(m)

soccer_list = zip(soccer_y, soccer_x)
for lat, lon in soccer_list:
    folium.CircleMarker(location = [lat, lon], 
                        fill_color = 'green',
                        radius = 5,
                        color='g',
                        fill = True,
                        fill_opacity = 0.7).add_to(m)
    

folium.LayerControl().add_to(m)
    
m.save('./html_maps/osm_soccer_pitches_centroids_' + cc +'.html')